# Evaluate Gradient Masking

This notebook runs some checks to determine if the networks may be gradient masked.
These checks may take some time and are not necessarily suitable as metrics themselves.

In [1]:
import argparse
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM
from advertorch.attacks import LinfSPSAAttack
from src.trainers import Trainer, FGSMTrainer
from robustbench.model_zoo.models import Carmon2019UnlabeledNet
from src.utils import adversarial_accuracy, fgsm_
import eagerpy as ep
from src.Nets import CIFAR_Wide_Res_Net, CIFAR_Res_Net, CIFAR_Net, ResNet18GradReg
from src.gradient_masking_tests import run_masking_benchmarks
%load_ext autoreload
%autoreload 2
%aimport src.Nets, src.trainers

## Setup to import trained CIFAR-10 Models

In [2]:
# setup
device = torch.device("cuda")
batch_size = 32
# remove the normalize
transform = transform = transforms.Compose(
            [transforms.ToTensor()]
)
        
normalized_min = (0 - 0.5) / 0.5
normalized_max = (1 - 0.5) / 0.5
train_dataset = datasets.CIFAR10(root='../data', train=True,
                                download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                  shuffle=True, num_workers=2)
test_dataset = datasets.CIFAR10(root='../data', train=False,
                               download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)
classes = classes = ('plane', 'car', 'bird', 'cat',
   'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load Trained Models

In [4]:
# # Regular CIFAR-10  ResNet Model
# model = CIFAR_Res_Net(device).eval()
# model.load_state_dict(torch.load("../models/cifar_res_net.model"))
# # CIFAR-10  ResNet Model trained with large FGSM steps
# fgsm_model = CIFAR_Wide_Res_Net(device).eval()
# fgsm_model.load_state_dict(torch.load("../models/cifar_res_net_fgsm06.model"))
# # CIFAR-10  ResNet Model trained with small FGSM steps
# fgsm_model_small = CIFAR_Res_Net(device).eval()
# fgsm_model_small.load_state_dict(torch.load("../models/cifar_res_net_20e_fgsm03.model"))
# # CIFAR-10  ResNet Model trained with large Step-ll steps
# step_ll_model = CIFAR_Res_Net(device).eval()
# step_ll_model.load_state_dict(torch.load("../models/cifar_res_net_20e_stepll.model"))
# CIFAR-10  ResNet Model trained through input gradient regularization
gradient_norm_model = CIFAR_Res_Net(device).eval()
state_dict = torch.load("D:/Libraries/Documents/tulip/cifar10/logs/cifar10/ResNet18/200e/best.pth.tar")['state_dict']
gradient_norm_model.load_state_dict(state_dict)

<All keys matched successfully>

# Run Benchmarks
### The benchmarks consist of:
- Standard Accuracy
- FGSM attack at different epsilons
- PGD attack
- Unbounded PGD attack
- Black Box (SPSA) Attack

For a gradient masked model, we might expect that:
- FGSM accuracy does not drop as epsilon increases
- PGD preforms worse than FGSM (turns out to not ever be the case)
- Unbounded PGD attack does not reach 0% accuracy
- Black box accuracy is lower than white box accuracy 

In [39]:
# Regular Model
run_masking_benchmarks(model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

  0%|                                                                                          | 0/313 [00:00<?, ?it/s]

Model accuracy: 84.86%


  0%|                                                                                          | 0/313 [00:00<?, ?it/s]

FGSM attack model accuracy -- eps = 0.006: 36.870000000000005%, eps = 0.03: 4.58%, eps = 0.06: 1.87%


  0%|                                                                                          | 0/313 [00:00<?, ?it/s]

PGD accuracy: 0.0%


  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

Unbounded PGD model accuracy: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.71s/it]

SPSA accuracy: 15.0%


Strangely, the fgsm model with large steps produces a model where increasing epsilon not only does not make the attack stronger, but instead, makes it much weaker.

In contrast, the model is significantly less robust at small epsilons.

It might be the case that we have created a subspace, beyond the decision boundary, making the model appear robust there.

However, this still does not explain how the model is now much more susceptible at smaller epsilons!

This seems to indicate the gradient is still informative?

In [40]:
# fgsm model with large steps
run_masking_benchmarks(fgsm_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.71s/it]

Model accuracy: 72.53%
FGSM attack model accuracy -- eps = 0.006: 10.299999999999999%, eps = 0.03: 73.95%, eps = 0.06: 85.24000000000001%
Gradient Masking Warning: Increasing epsilon did not improve the attack!!
PGD accuracy: 0.0%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 5.0%
Gradient Masking Warning: Black Box attack was stronger than FGSM attack!!


In [10]:
# fgsm model with small steps
run_masking_benchmarks(fgsm_model_small, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.86s/it]

Model accuracy: 69.61%
FGSM attack model accuracy -- eps = 0.006: 62.53999999999999%, eps = 0.03: 36.83%, eps = 0.06: 16.36%
PGD accuracy: 7.1%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 45.0%


### Step-ll potential gradient masking!
We notice that increasing epsilon does not improve the attack. 

However, unlike with FGSM, the model is more robust at small epsilons.

This together with black box outpreforming white box attacks, suggests an incidence of gradient masking

In [13]:
# step-ll model
run_masking_benchmarks(step_ll_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.76s/it]

Model accuracy: 72.36%
FGSM attack model accuracy -- eps = 0.006: 54.31%, eps = 0.03: 56.74%, eps = 0.06: 55.71%
Gradient Masking Warning: Increasing epsilon did not improve the attack!!
PGD accuracy: 0.4%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 17.0%
Gradient Masking Warning: Black Box attack was stronger than FGSM attack!!


Gradient normalization model does become more robust.

Tried with 200 epochs, as in paper. all other nets 20e.

In [5]:
run_masking_benchmarks(gradient_norm_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.72s/it]

Model accuracy: 92.29%
FGSM attack model accuracy -- eps = 0.006: 71.89999999999999%, eps = 0.03: 13.489999999999998%, eps = 0.06: 3.5000000000000004%
PGD accuracy: 0.0%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 9.0%


## Wide RESNET Model

In [7]:
run_masking_benchmarks(wide_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [03:41<00:00, 17.02s/it]

Model accuracy: 84.27%
FGSM attack model accuracy -- eps = 0.006: 31.900000000000002%, eps = 0.03: 5.33%, eps = 0.06: 4.5600000000000005%
PGD accuracy: 0.0%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 11.0%


In [8]:
run_masking_benchmarks(wide_fgsm_model, test_dataset, epsilon=0.06, device=device, batch_size=batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [03:34<00:00, 16.49s/it]

Model accuracy: 63.190000000000005%
FGSM attack model accuracy -- eps = 0.006: 12.4%, eps = 0.03: 72.77%, eps = 0.06: 85.47%
Gradient Masking Warning: Increasing epsilon did not improve the attack!!
PGD accuracy: 0.0%
Unbounded PGD model accuracy: 0.0%
SPSA accuracy: 0.0%
Gradient Masking Warning: Black Box attack was stronger than FGSM attack!!
